# Forecasting Household Electric Power Consumption
---

## Table of Contents
---

1. [Introduction](#introduction)

2. [Libraries](#libraries)

3. [Data Sets](#dataSets)

    -[UCI Data Set](#uci-data-set)

    -[UCI Dataset Dictionary](#uci-dataset-dictionary)

    -[UCI Dataset Datatype Formating](#uci-dataset-datatype-formating)

    -[Weather Data Set](#weather-data-set)

    -[Weather Data Dictionary](#weather-data-dictionary)

    -[Weather Data Formating](#weather-data-formating)

4. [Combining DataSets](#combining-datasets)

5. [Data Cleaning](#data-cleaning)

6. [Saving Combined and Cleaned DataSet](#saving-combined-and-cleaned-dataset)

7. [Next Steps:EDA](#next-steps-eda)







[def]: uci

# Introduction
---



The goal of this project is to build a machine learning model to accurately predict theforecast energy usage and uncover patterns in electricity consumption. This will notebook explores the integration of  Individual Household Electric Power Consumption dataset with  weather dataset through an API



# Libraries <a class="anchor" id="Libraries"></a>
---

In [1]:
# Installing the ucimlrepo package
# This package provides access to the UCI Machine Learning Repository datasets
# Need to run this in a Jupyter notebook if not installed in the environment
# or if you want to ensure the latest version is installed.
!pip install ucimlrepo

In [2]:
# Importing necessary libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from warnings import filterwarnings
from datetime import datetime
import warnings
import requests
warnings.filterwarnings('ignore')
import ucimlrepo
from ucimlrepo import fetch_ucirepo 

# Data Sets 
---

For this project, we will be using the UCI Individual Household Electric Power Consumption dataset
The Data set contains measurements of electric power consumption in one household with a one-minute sampling rate over a period of almost 4 years. Different electrical quantities and some sub-metering values are available

## UCI Data Set



In [3]:
# Loading the dataset into a dataframe, this might take a while
# as it fetches the data from the UCI repository estimated to take 2-3 minutes on first run
# fetch dataset 
individual_household_electric_power_consumption = fetch_ucirepo(id=235) 
power_df=individual_household_electric_power_consumption.data.original
power_df.head()

,Date,Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
0,16/12/2006,17:24:00,4.216,0.418,234.840,18.400,0.000,1.000,17.0
1,16/12/2006,17:25:00,5.360,0.436,233.630,23.000,0.000,1.000,16.0
2,16/12/2006,17:26:00,5.374,0.498,233.290,23.000,0.000,2.000,17.0
3,16/12/2006,17:27:00,5.388,0.502,233.740,23.000,0.000,1.000,17.0
4,16/12/2006,17:28:00,3.666,0.528,235.680,15.800,0.000,1.000,17.0


## UCI Dataset Dictionary



| **Feature Name**         | **Description**                                                                                                        | **Unit**            |
|--------------------------|------------------------------------------------------------------------------------------------------------------------|---------------------|
| `date`                   | The calendar date of the observation, in `dd/mm/yyyy` format.                                                          | —                   |
| `time`                   | The time of the observation, in `hh:mm:ss` format.                                                                     | —                   |
| `global_active_power`    | Total active power consumed by the household, averaged per minute. Active power is the actual energy used.            | kilowatt (kW)       |
| `global_reactive_power`  | Reactive power consumed — energy that cycles between source and load but is not used. Important for grid stability.   | kilowatt (kW)       |
| `voltage`                | Average voltage of the household circuit during the minute.                                                            | volt (V)            |
| `global_intensity`       | Average current drawn by the household during that minute.                                                             | ampere (A)          |
| `sub_metering_1`         | Energy used by kitchen appliances (e.g., dishwasher, oven, microwave — hot plates use gas).                           | watt-hour (Wh/min)  |
| `sub_metering_2`         | Energy used by laundry room appliances (e.g., washer, dryer, fridge, light).                                           | watt-hour (Wh/min)  |
| `sub_metering_3`         | Energy used by electric water heater and air conditioner.                                                              | watt-hour (Wh/min)  |


In [4]:
power_df.tail()

,Date,Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
2075254,26/11/2010,20:58:00,0.946,0.0,240.43,4.0,0.0,0.0,0.0
2075255,26/11/2010,20:59:00,0.944,0.0,240.0,4.0,0.0,0.0,0.0
2075256,26/11/2010,21:00:00,0.938,0.0,239.82,3.8,0.0,0.0,0.0
2075257,26/11/2010,21:01:00,0.934,0.0,239.7,3.8,0.0,0.0,0.0
2075258,26/11/2010,21:02:00,0.932,0.0,239.55,3.8,0.0,0.0,0.0


In [5]:
power_df.shape
# Displaying the columns of the dataframe

(2075259, 9)

There are 2,075,259 rows and 5 columns in this dataset

In [6]:
# Next lets check the data types and formats of the  columns in this data set using the .info() function
power_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075259 entries, 0 to 2075258
Data columns (total 9 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   Date                   object 
 1   Time                   object 
 2   Global_active_power    object 
 3   Global_reactive_power  object 
 4   Voltage                object 
 5   Global_intensity       object 
 6   Sub_metering_1         object 
 7   Sub_metering_2         object 
 8   Sub_metering_3         float64
dtypes: float64(1), object(8)
memory usage: 142.5+ MB


This data set contains the following data types:

(1) float64 variable and (8) object variable

We need to change all objects to float except for Date and Time, which should be in Datetime

## UCI Dataset Datatype Formating

In [7]:

#Fixing the data types of the columns

#Splitting the 'Date' and 'Time' columns into a single 'DateTime' column
power_df['DateTime'] = pd.to_datetime(power_df['Date'] + ' ' + power_df['Time'])
power_df=power_df.drop(columns=['Date', 'Time'])
power_df=power_df.set_index('DateTime')
power_df= power_df.reset_index()

#Changing  the object columns to float
# Convert all columns except 'DateTime' to float
for col in power_df.columns:
    if col != 'DateTime':
        power_df[col] = pd.to_numeric(power_df[col], errors='coerce')


In [8]:
power_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075259 entries, 0 to 2075258
Data columns (total 8 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   DateTime               datetime64[ns]
 1   Global_active_power    float64       
 2   Global_reactive_power  float64       
 3   Voltage                float64       
 4   Global_intensity       float64       
 5   Sub_metering_1         float64       
 6   Sub_metering_2         float64       
 7   Sub_metering_3         float64       
dtypes: datetime64[ns](1), float64(7)
memory usage: 126.7 MB


## Weather Data Set  

In [9]:

# Set coordinates and time range
params = {
    "latitude": 48.7766,
    "longitude": 2.2905,
    "start_date": "2006-12-16",
    "end_date": "2010-11-26",
    "hourly": [
        "temperature_2m", "relative_humidity_2m", "dew_point_2m",
        "apparent_temperature", "surface_pressure", "cloudcover",
        "windspeed_10m", "windgusts_10m", "winddirection_10m",
        "shortwave_radiation", "direct_radiation", "diffuse_radiation"
    ],
    "timezone": "Europe/Paris"
}

# Fetch the data
response = requests.get("https://archive-api.open-meteo.com/v1/archive", params=params)
weather_data = response.json()

# Convert to DataFrame
weather_hourly_df = pd.DataFrame(weather_data['hourly'])
weather_hourly_df['Datetime'] = pd.to_datetime(weather_hourly_df['time'])
# Drop the 'time' column
weather_hourly_df.drop(columns=['time'], inplace=True)
weather_hourly_df.set_index('Datetime', inplace=True)

# Reset index to merge with power_df

weather_hourly_df.reset_index()



,Datetime,temperature_2m,relative_humidity_2m,dew_point_2m,apparent_temperature,surface_pressure,cloudcover,windspeed_10m,windgusts_10m,winddirection_10m,shortwave_radiation,direct_radiation,diffuse_radiation
0,2006-12-16 00:00:00,5.4,91,4.1,1.3,1011.8,1,18.4,31.7,203,0.0,0.0,0.0
1,2006-12-16 01:00:00,5.3,92,4.2,1.2,1011.9,8,18.6,32.0,216,0.0,0.0,0.0
2,2006-12-16 02:00:00,5.1,93,4.0,1.1,1012.1,63,17.3,31.3,222,0.0,0.0,0.0
3,2006-12-16 03:00:00,5.1,93,4.1,1.4,1011.8,92,16.1,29.9,220,0.0,0.0,0.0
4,2006-12-16 04:00:00,5.3,94,4.3,1.6,1011.6,100,15.9,29.9,219,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
34603,2010-11-26 19:00:00,1.0,75,-3.0,-2.5,995.0,22,6.7,11.5,234,0.0,0.0,0.0
34604,2010-11-26 20:00:00,-0.1,79,-3.2,-3.7,994.9,20,6.9,11.5,223,0.0,0.0,0.0
34605,2010-11-26 21:00:00,-1.2,91,-2.5,-4.8,994.7,38,7.5,13.0,215,0.0,0.0,0.0
34606,2010-11-26 22:00:00,-1.3,91,-2.6,-5.0,994.4,74,8.0,13.0,207,0.0,0.0,0.0


In [10]:
weather_hourly_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 34608 entries, 2006-12-16 00:00:00 to 2010-11-26 23:00:00
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   temperature_2m        34608 non-null  float64
 1   relative_humidity_2m  34608 non-null  int64  
 2   dew_point_2m          34608 non-null  float64
 3   apparent_temperature  34608 non-null  float64
 4   surface_pressure      34608 non-null  float64
 5   cloudcover            34608 non-null  int64  
 6   windspeed_10m         34608 non-null  float64
 7   windgusts_10m         34608 non-null  float64
 8   winddirection_10m     34608 non-null  int64  
 9   shortwave_radiation   34608 non-null  float64
 10  direct_radiation      34608 non-null  float64
 11  diffuse_radiation     34608 non-null  float64
dtypes: float64(9), int64(3)
memory usage: 3.4 MB


## Weather Data Dictionary

| **Feature Name**        | **Description**                                                        | **Unit**           |
|--------------------------|------------------------------------------------------------------------|--------------------|
| `temperature_2m`         | Air temperature measured at 2 meters above ground                      | °C (Celsius)       |
| `dew_point_2m`           | Temperature at which air becomes saturated with moisture               | °C (Celsius)       |
| `apparent_temperature`   | "Feels-like" temperature considering wind and humidity                  | °C (Celsius)       |
| `relative_humidity_2m`   | Relative humidity at 2 meters above ground                              | % (percent)        |
| `cloudcover`             | Total cloud cover fraction                                              | % (percent)        |
| `windspeed_10m`          | Mean wind speed at 10 meters above ground                               | m/s (meters/second)|
| `windgusts_10m`          | Maximum wind gust speed at 10 meters                                    | m/s (meters/second)|
| `winddirection_10m`      | Wind direction (0° = North, 90° = East, etc.)                           | ° (degrees)        |
| `shortwave_radiation`    | Total shortwave solar radiation reaching the surface                    | W/m²               |
| `direct_radiation`       | Direct sunlight reaching the surface without scattering                 | W/m²               |
| `diffuse_radiation`      | Scattered sunlight (diffused through atmosphere) reaching the surface   | W/m²               |
| `surface_pressure`       | Atmospheric pressure at surface level                                   | hPa (hectopascals) |


## Weather Data Formating

In [11]:

# Resample weather_hourly_df to minute frequency and interpolate
weather_minute_df = weather_hourly_df.resample('1T').interpolate()



# Combining DataSets

In [12]:

df_merged = pd.merge(
    power_df,
    weather_minute_df.reset_index(),
    left_on='DateTime',
    right_on='Datetime',
    how='inner'
)

In [13]:
df_merged.drop(columns=['Datetime'], inplace=True)
df_merged

,DateTime,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3,temperature_2m,relative_humidity_2m,dew_point_2m,apparent_temperature,surface_pressure,cloudcover,windspeed_10m,windgusts_10m,winddirection_10m,shortwave_radiation,direct_radiation,diffuse_radiation
0,2006-12-16 17:24:00,4.216,0.418,234.84,18.4,0.0,1.0,17.0,7.120000,94.200000,6.240000,4.100000,1012.400000,100.0,14.280000,31.100,329.400000,8.000000,0.0,8.000000
1,2006-12-16 17:25:00,5.360,0.436,233.63,23.0,0.0,1.0,16.0,7.108333,94.166667,6.225000,4.091667,1012.416667,100.0,14.241667,31.075,329.416667,7.833333,0.0,7.833333
2,2006-12-16 17:26:00,5.374,0.498,233.29,23.0,0.0,2.0,17.0,7.096667,94.133333,6.210000,4.083333,1012.433333,100.0,14.203333,31.050,329.433333,7.666667,0.0,7.666667
3,2006-12-16 17:27:00,5.388,0.502,233.74,23.0,0.0,1.0,17.0,7.085000,94.100000,6.195000,4.075000,1012.450000,100.0,14.165000,31.025,329.450000,7.500000,0.0,7.500000
4,2006-12-16 17:28:00,3.666,0.528,235.68,15.8,0.0,1.0,17.0,7.073333,94.066667,6.180000,4.066667,1012.466667,100.0,14.126667,31.000,329.466667,7.333333,0.0,7.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2075254,2010-11-26 20:58:00,0.946,0.000,240.43,4.0,0.0,0.0,0.0,-1.163333,90.600000,-2.523333,-4.763333,994.706667,37.4,7.480000,12.950,215.266667,0.000000,0.0,0.000000
2075255,2010-11-26 20:59:00,0.944,0.000,240.00,4.0,0.0,0.0,0.0,-1.181667,90.800000,-2.511667,-4.781667,994.703333,37.7,7.490000,12.975,215.133333,0.000000,0.0,0.000000
2075256,2010-11-26 21:00:00,0.938,0.000,239.82,3.8,0.0,0.0,0.0,-1.200000,91.000000,-2.500000,-4.800000,994.700000,38.0,7.500000,13.000,215.000000,0.000000,0.0,0.000000
2075257,2010-11-26 21:01:00,0.934,0.000,239.70,3.8,0.0,0.0,0.0,-1.201667,91.000000,-2.501667,-4.803333,994.695000,38.6,7.508333,13.000,214.866667,0.000000,0.0,0.000000


In [14]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075259 entries, 0 to 2075258
Data columns (total 20 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   DateTime               datetime64[ns]
 1   Global_active_power    float64       
 2   Global_reactive_power  float64       
 3   Voltage                float64       
 4   Global_intensity       float64       
 5   Sub_metering_1         float64       
 6   Sub_metering_2         float64       
 7   Sub_metering_3         float64       
 8   temperature_2m         float64       
 9   relative_humidity_2m   float64       
 10  dew_point_2m           float64       
 11  apparent_temperature   float64       
 12  surface_pressure       float64       
 13  cloudcover             float64       
 14  windspeed_10m          float64       
 15  windgusts_10m          float64       
 16  winddirection_10m      float64       
 17  shortwave_radiation    float64       
 18  direct_radiation      

In [15]:

#Making sure all columns are rounded to 3 decimal places
df_merged=df_merged.round(3)

In [16]:
df_merged.head()

,DateTime,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3,temperature_2m,relative_humidity_2m,dew_point_2m,apparent_temperature,surface_pressure,cloudcover,windspeed_10m,windgusts_10m,winddirection_10m,shortwave_radiation,direct_radiation,diffuse_radiation
0,2006-12-16 17:24:00,4.216,0.418,234.84,18.4,0.0,1.0,17.0,7.120,94.200,6.240,4.100,1012.400,100.0,14.280,31.100,329.400,8.000,0.0,8.000
1,2006-12-16 17:25:00,5.360,0.436,233.63,23.0,0.0,1.0,16.0,7.108,94.167,6.225,4.092,1012.417,100.0,14.242,31.075,329.417,7.833,0.0,7.833
2,2006-12-16 17:26:00,5.374,0.498,233.29,23.0,0.0,2.0,17.0,7.097,94.133,6.210,4.083,1012.433,100.0,14.203,31.050,329.433,7.667,0.0,7.667
3,2006-12-16 17:27:00,5.388,0.502,233.74,23.0,0.0,1.0,17.0,7.085,94.100,6.195,4.075,1012.450,100.0,14.165,31.025,329.450,7.500,0.0,7.500
4,2006-12-16 17:28:00,3.666,0.528,235.68,15.8,0.0,1.0,17.0,7.073,94.067,6.180,4.067,1012.467,100.0,14.127,31.000,329.467,7.333,0.0,7.333


# Data Cleaning

------------------------------------------------------------------------------------------------

In [17]:
# Check for missing values
df_merged.isna().sum()

DateTime                     0
Global_active_power      25979
Global_reactive_power    25979
Voltage                  25979
Global_intensity         25979
Sub_metering_1           25979
Sub_metering_2           25979
Sub_metering_3           25979
temperature_2m               0
relative_humidity_2m         0
dew_point_2m                 0
apparent_temperature         0
surface_pressure             0
cloudcover                   0
windspeed_10m                0
windgusts_10m                0
winddirection_10m            0
shortwave_radiation          0
direct_radiation             0
diffuse_radiation            0
dtype: int64

In [18]:

# It's also useful to calculate the proportion of nulls missing
df_merged.isna().mean() * 100

DateTime                 0.000000
Global_active_power      1.251844
Global_reactive_power    1.251844
Voltage                  1.251844
Global_intensity         1.251844
Sub_metering_1           1.251844
Sub_metering_2           1.251844
Sub_metering_3           1.251844
temperature_2m           0.000000
relative_humidity_2m     0.000000
dew_point_2m             0.000000
apparent_temperature     0.000000
surface_pressure         0.000000
cloudcover               0.000000
windspeed_10m            0.000000
windgusts_10m            0.000000
winddirection_10m        0.000000
shortwave_radiation      0.000000
direct_radiation         0.000000
diffuse_radiation        0.000000
dtype: float64

 Have about ~1% of the data missing in the missing columns obtained from the dataset, we will drop them since they seem to be insignificant ~ 25000 vs 2,075,259

In [19]:
# Handle missing values
df_merged = df_merged.dropna()

In [20]:
df_merged.isna().sum()

DateTime                 0
Global_active_power      0
Global_reactive_power    0
Voltage                  0
Global_intensity         0
Sub_metering_1           0
Sub_metering_2           0
Sub_metering_3           0
temperature_2m           0
relative_humidity_2m     0
dew_point_2m             0
apparent_temperature     0
surface_pressure         0
cloudcover               0
windspeed_10m            0
windgusts_10m            0
winddirection_10m        0
shortwave_radiation      0
direct_radiation         0
diffuse_radiation        0
dtype: int64

In [21]:
# We can sum the resultant array to get the total number of duplicate rows
df_merged.duplicated().sum()

0

No duplicated rows are found in the dataset. 



# Saving Combined and Cleaned DataSet
---

This dataset has  been cleaned and loaded into a consitent format to do EDA now, for this first we will save this data set to a csv file, which can be used for the following sections of EDA, Feature engineering and then modelling. 

In [22]:
# Saving file to csv

# Specify the file path where you want to save the CSV file
file_path_save=r'C:\Users\karni\OneDrive\Bootcamp\Projects\Capstone\Brainstation Capstone Project\Appliance Energy Usage\data\power_weather.csv'

df_merged.to_csv(file_path_save, index=False)



# Next Steps: EDA
---

 Now that we have the merged dataset, we can proceed with Exploratory Data Analysis (EDA) and further analysis as needed.We can visualize the data, check for trends, and perform any other analysis required.